In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import time

In [ ]:
# 公司清單
df = pd.read_csv('data/tse_listed_stocks.csv')
df['code'] = df['code'].astype(str)

# 刪除部分股票代碼
drop_list = ['6909.TW', '7736.TW', '7769.TW', '7711.TW', '7780.TW', '6838.TW', '7799.TW', '6589.TW', '6936.TW', 
            '3150.TW', '6885.TW', '8162.TW', '7786.TW', '6918.TW', '6862.TW', '6962.TW', '7750.TW', '4585.TW', '4949.TW', 
            '7732.TW', '3716.TW', '6831.TW', '2646.TW', '6969.TW', '7749.TW', '4441.TW', '6914.TW', '7791.TW', '6887.TW', 
            '1563.TW', '6771.TW', '7765.TW', '6957.TW', '6949.TW', '7705.TW', '7730.TW', '4771.TW', '2248.TW', '7740.TW', 
            '6965.TW', '3717.TW', '6614.TW', '6944.TW', '6988.TW', '6890.TW', '6794.TW', '6928.TW', '6924.TW', '6923.TW', 
            '8487.TW', '6906.TW', '7788.TW', '2762.TW', '7610.TW', '7722.TW','9103.TW','910322.TW','9105.TW','910861.TW',
            '9110.TW','911608.TW','911622.TW' ,'911868.TW','912000.TW','9136.TW']

drop_list_clean = [x.replace('.TW', '') for x in drop_list]
drop_list = drop_list_clean
drop_list = pd.DataFrame(drop_list, columns=['code'])
df['code'] = df['code'].astype(str).str.strip()
df = df[~df['code'].isin(drop_list['code'])]

In [3]:
# choose tickers 
all_codes = df['code'].tolist()
tickers = [f"{c}.TW" for c in df['code'].astype(str).tolist()]

# yfinance 下載
start = "2015-01-01"
end = "2025-11-25"

batch_size = 50
all_data = {}

for i in range(0, len(tickers), batch_size):
    batch = tickers[i:i+batch_size]
    print(f"Downloading batch {i} ~ {i+len(batch)}: {batch}")
    
    data = yf.download(
        batch,
        start=start,
        end=end,
        auto_adjust=True,
        group_by='ticker'
    )
    
    # 這裡才是關鍵：把每 batch 的資料 append，而不是覆蓋
    for t in batch:
        if t in data.columns.levels[0]:  # 該股票下載成功
            all_data[t] = data[t]
    
    time.sleep(2)  # 避免被 Yahoo 封鎖

# 最後把 all_data 合併成一個 MultiIndex DataFrame
px1 = pd.concat(all_data, axis=1)

[*********************100%***********************]  50 of 50 completed


[*********************100%***********************]  50 of 50 completed


[*********************100%***********************]  50 of 50 completed


[*********************100%***********************]  50 of 50 completed


[*********************100%***********************]  50 of 50 completed


...

[*********************100%***********************]  50 of 50 completed


[*********************100%***********************]  50 of 50 completed


[*********************100%***********************]  50 of 50 completed


[*********************100%***********************]  50 of 50 completed


[*********************100%***********************]  6 of 6 completed


In [4]:
px1.to_pickle("tse_px1.pkl")  # 建議存成 pickle，不會亂掉

In [5]:
px1.columns = px1.columns.set_names(['Ticker', 'Price'])

# 資料整理為長格式
px_long = px1.stack(level='Ticker').reset_index()
px_long.columns = [col.lower() for col in px_long.columns]
px_long = px_long.rename(columns={'date': 'date'})
px_long = px_long.sort_values(['ticker', 'date']).reset_index(drop=True)
px_long['returns'] = px_long.groupby('ticker')['close'].pct_change()

#px_long.dropna(subset=['returns'], inplace=True) # 移除所有 returns 欄位為 NaN 的行
px_long.to_csv('stock_data.csv',index=False)
px_long

/var/folders/gk/tqz928vx1m96n53nylzj1y800000gn/T/ipykernel_67150/512525780.py:4: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  px_long = px1.stack(level='Ticker').reset_index()


,date,ticker,open,high,low,close,volume,returns
0,2015-01-05,1101.TW,19.371103,19.371103,19.103300,19.192568,3847400.0,NaN
1,2015-01-06,1101.TW,18.969396,19.014031,18.790860,18.857811,10386622.0,-0.017442
2,2015-01-07,1101.TW,18.857812,18.991714,18.679277,18.790861,11841293.0,-0.003550
3,2015-01-08,1101.TW,18.857811,19.036347,18.790860,18.857811,11815462.0,0.003563
4,2015-01-09,1101.TW,18.880131,19.014034,18.656961,18.656961,14240820.0,-0.010651
...,...,...,...,...,...,...,...,...
2503072,2025-11-18,9958.TW,133.000000,134.500000,129.500000,130.500000,2817821.0,-0.022472
2503073,2025-11-19,9958.TW,131.000000,133.500000,129.000000,130.500000,1693272.0,0.000000
2503074,2025-11-20,9958.TW,132.000000,136.000000,131.500000,135.000000,1580122.0,0.034483
2503075,2025-11-21,9958.TW,132.500000,135.000000,131.000000,135.000000,1293210.0,0.000000
